In [1]:
import nltk
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize

In [2]:
data_frame = pd.read_csv('/mnt/workspace/openasl-pre/openasl-v1.0.tsv', sep='\t')

# Load training text samples
split = 'train'
column = 'raw-text'
data_frame = data_frame.loc[data_frame['split'].str.contains(split)]
translation = data_frame[column]
vids = data_frame['vid']

# Using Punkt to tokenize words
sent_tks = [word_tokenize(s.lower()) for s in translation]
tag_res = [nltk.pos_tag(tks) for tks in sent_tks]

joined_tag_res = []
for l in tag_res:
    joined_tag_res.extend(l)

freq_dist = nltk.ConditionalFreqDist(joined_tag_res)

exclude_words = ['was', 'i', 'said', 'aslcaptions.com', '\'s', 'is', 'be', 'are', 'has', 'www.aslcaptions.com', 'did', '\'ve', '\'m', '%', 've', 'r', 'd', '*', 'b', 'ed', 'e.', '[', ']', 'dpan.tv', 'iii', '<', '>', '/i', '']
collect_keys = {'NN', 'NNP', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
filtered_words = {}
for word, freq in freq_dist.items():
    if word in exclude_words: continue
    key_set = set(freq.keys())
    intersect = collect_keys.intersection(key_set)
    if len(intersect) > 0:
        filtered_freq = {}
        for tag in intersect:
            if freq[tag] > 10:
                filtered_freq[tag] = freq[tag]
        if len(filtered_freq) > 0: 
            filtered_words[word] = filtered_freq

# Load GloVe embeddings
# vocab = []
# embeddings = []
# with open('/mnt/workspace/slt_baseline/notebooks/glove/glove.6B.300d.txt', 'r') as f:
#     for line in f:
#         items = line.strip().split(' ')
#         vocab.append(items[0])
#         embeddings.append(np.asarray(items[1:], 'float32'))


/home/pai/envs/slt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import json

with open('openasl-v1.0/uncased_filtred_VNs.json', 'w') as f:
    json.dump(filtered_words, f)

In [4]:
# Filter cross filter with glove vocabulary
import numpy as np
vocab = []
embeddings = []
with open('/mnt/workspace/slt_baseline/notebooks/glove/glove.6B.300d.txt', 'r') as f:
    for line in f:
        items = line.strip().split(' ')
        vocab.append(items[0])
        embeddings.append(np.asarray(items[1:], 'float32'))

In [5]:
VN_dict = json.load(open('openasl-v1.0/uncased_filtred_VNs.json', 'r'))
VNs = VN_dict.keys()

OOV = 0
OOV_word = []
for vn in VNs:
    if vn not in vocab:
        OOV += 1
        OOV_word.append(vn)

In [6]:
for k in OOV_word:
    stat = VN_dict[k]
    total = 0
    for pos, num in stat.items():
        total += num
    print(total, k)

57 guyger
15 renca
13 taloali
47 convo
285 mavrick
17 dcara
49 vaping
18 vindman
357 covid-19
21 sdsd
88 ylc
50 gofundme
33 breonna
23 hydroxychloroquine
13 yovanovitch
20 dpan
33 ntid
13 ntid/rit
23 burisma
57 cordano
12 aasd
35 ehdi
15 deafverse
44 covid
17 ocasio-cortez
14 guaido
13 hcq
13 h.w
18 fsdb
36 audism
21 brexit
25 feimer
50 sondland
38 zelensky
12 besgrove
12 retweeted
26 selfie
55 arbery
12 bipoc
28 nbda
25 tlaib
29 bolsonaro
11 faceapp
12 captioner
11 rayshard
11 emoji
27 tiktok
2401 sign1news
515 wooddall
23 drejka
26 selfies
16 e-cigarettes
17 berreth
15 emojis
12 ochoa-lopez
12 blasey
12 kaytee
14 deblasio
11 sayoc
34 maleah
17 remdesivir
11 mssd
12 daca
11 asdc
12 paffumi
14 deaf/hard
12 d'oeil
11 fortnite
11 deafspace
20 dwu
25 kpta
11 vallow
11 captioners
15 ip-cts
23 nltc
11 re-authorization
16 nbsp
24 nad.org
32 h.r
14 dejoy
39 crpd
12 schnatter


In [7]:
for k in OOV_word:
    VN_dict.pop(k)

with open('openasl-v1.0/uncased_filtred_glove_VNs.json', 'w') as f:
    json.dump(VN_dict, f)

In [8]:
# Generate Index word mapping for glove filtered VNs
import json

with open('openasl-v1.0/uncased_filtred_glove_VNs.json', 'r') as f:
    vn_dict = json.load(f)

vn_words = list(vn_dict.keys())
with open('openasl-v1.0/uncased_filtred_glove_VN_idxs.txt', 'w') as f:
    for idx, word in enumerate(vn_words):
        f.write(f'{idx} {word}\n')

In [9]:
# Generate the corresponding embedding pkl
import numpy as np

vn_glove_embeddings = []

glove_embedding_dict = {}

with open('/mnt/workspace/slt_baseline/notebooks/glove/glove.6B.300d.txt', 'r') as f:
    for line in f:
        items = line.strip().split(' ')
        glove_embedding_dict[items[0]] = np.asarray(items[1:], 'float32')
        
for word in vn_words:
    vn_glove_embeddings.append(glove_embedding_dict[word])

In [10]:
vn_glove_embed = np.stack(vn_glove_embeddings, axis=0)
vn_glove_embed.shape

(5523, 300)

In [ ]:
import pickle as pkl
with open('openasl-v1.0/uncased_filtred_glove_VN_embed.pkl', 'wb') as f:
    pkl.dump(vn_glove_embed, f)

In [ ]:
# Generate trainning infomation
data_frame = pd.read_csv('/mnt/workspace/openasl-pre/openasl-v1.0.tsv', sep='\t')

# Load training text samples
split = 'train'
column = 'raw-text'
data_frame = data_frame.loc[data_frame['split'].str.contains(split)]
translation = data_frame[column]
vids = data_frame['vid']

VN_dict = json.load(open('openasl-v1.0/uncased_filtred_glove_VNs.json', 'r'))
VNs = VN_dict.keys()

matched = {}
for vid, trans in zip(vids, translation):
    ref_word_list = word_tokenize(trans)
    matched_words = []
    for ref_word in ref_word_list:
        if ref_word in VNs:
            matched_words.append(ref_word)
    matched[vid] = matched_words

with open('openasl-v1.0/uncased_filtred_glove_VN_matched_train.json', 'w') as f:
    json.dump(matched, f)